# **Template OP on salish**

In [9]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [10]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [11]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coordsWW3'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))

## Definitions

In [12]:
start = datetime(2018, 12, 23) #Start date
# Set Time length [days] and timestep [seconds]
length = 10
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 5 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [13]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 960)             # desnsity PE
    size = Variable('size', initial = 5e-5)

In [14]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [15]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [16]:
#Set start date time and the name of the output file
name = 'Waves1' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/Waves120181223_1n_20190102_1n.nc


## Simulation

In [17]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

#dimensions = {'lon': coord['longitude'], 'lat': coord['latitude'], 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [18]:
def Waves(particle, fieldset, time):
    '''Stokes drift'''
 
    z = particle.depth
    us0 = fieldset.lm[time,particle.depth, particle.lat, particle.lon]
    vs0 = fieldset.vuss[time, particle.depth, particle.lat, particle.lon]
    wl = fieldset.lm[time, particle.depth, particle.lat, particle.lon]
    k = (2*math.pi)/wl
    us = us0*math.exp(-2*k*z)
    vs = vs0*math.exp(-2*k*z)
    #print(z)
    particle.lon += us * particle.dt
    particle.lat += vs * particle.dt

In [19]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

p_flt = pset.Kernel(Waves) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D + p_flt, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayJITParticleAdvectionRK4_3DWaves ==> /tmp/parcels-2894/lib8871b56d26de9cfcb3aa35a975fd2595_0.so


Correct cell not found for (-121.826065, 49.188363) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 485 215
            new particle indices: (yi, xi) 337 173
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 3.4610165194962792e+02 3.6231202046995348e-01
Correct cell not found for (-121.566622, 49.187915) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 472 242
            new particle indices: (yi, xi) 318 200
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 3.5710999034554106e+02 -1.4341112190830357e-01
Correct cell not found for (-122.656551, 49.187016) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 485 214
            new particle indices: (yi, xi) 396 172
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 2.2585588468803695e+02 4.3806680557991606e-01
Correct cell not found for (-122.648102, 49.189740) after 10

INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-FCMQKUMC.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-FCMQKUMC" to convert these to a NetCDF file during the run.
N/A% (0 of 864000.0) |                   | Elapsed Time: 0:00:00 ETA:  --:--:--

Correct cell not found for (-121.445264, 49.306932) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 456 337
            new particle indices: (yi, xi) 334 295
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 2.9524793222747098e+02 9.3413451174809681e-01
Correct cell not found for (-122.190771, 49.308976) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 457 337
            new particle indices: (yi, xi) 388 295
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 1.8668479728048342e+02 1.2440346574082717e-01
Particle 233 lost !! [-120.4673440280543, 49.308244276712784, 20.68464359278759, 22140.0]
Particle 238 lost !! [-121.44526429347081, 49.30693168437554, 19.353986378229486, 21600.0]
Particle 247 lost !! [-118.1696729112582, 49.30690620234163, 20.027179233258693, 21870.0]
Particle 288 lost !! [-122.19077076553067, 49.30897576774628, 21.59222328807932, 22680.0]
Particle 299 

  3% (30600.0 of 864000.0) |             | Elapsed Time: 0:00:08 ETA:   0:07:47

...

  3% (34200.0 of 864000.0) |             | Elapsed Time: 0:00:10 ETA:   0:09:07

Particle 113 lost !! [-99.13514464787053, 50.064022626145004, 15.324388929757305, 40590.0]
Particle 157 lost !! [617.2604090871782, 51.10344557559699, 5.556569159246841, 40410.0]


 37% (327600.0 of 864000.0) |####        | Elapsed Time: 0:03:14 ETA:   0:05:35

Correct cell not found for (-121.944312, 49.293311) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 454 334
            new particle indices: (yi, xi) 367 292
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) 2.2363694437103706e+02 6.3903882513044763e-01
Particle 441 lost !! [-121.94431204476794, 49.29331088431841, 20.016414315807197, 334620.0]


100% (864000.0 of 864000.0) |############| Elapsed Time: 0:08:49 Time:  0:08:49
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
